In [1]:
!pip install beautifulsoup4

In [39]:
!pip install lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 4.9 MB/s eta 0:00:00 MB/s eta 0:00:01:01


In [2]:
import requests
from bs4 import BeautifulSoup

# URL of the Standard Meteorological Dataset page
url = "https://www.ndbc.noaa.gov/historical_data.shtml"

# Make a GET request to the URL
response = requests.get(url)

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# # Find all the links to station data
# station_links = soup.find_all('a', href=lambda href: href.startswith('/view_data.php?station='))

# # Extract the station ID from each link
# station_ids = [link['href'].split('=')[1] for link in station_links]

# # Print the station IDs and links
# for station_id, link in zip(station_ids, station_links):
#     print(f"Station ID: {station_id}, Link: {link}")

In [3]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<title>NDBC - Historical NDBC Data</title>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="What's New at the NDBC Web Site?" name="DC.title"/>
<meta content="The National Data Buoy Center's page for new events." name="DC.description"/>
<meta content="weather, buoy, weather buoy, marine, forecast, hurricane, wind, wave, offshore, surfing, temperature, meteorology, climate, ocean" name="DC.subject"/>
<meta content="US Department of Commerce, National Oceanic and Atmospheric Administration, National Weather Service, National Data Buoy Center" name="DC.creator"/>
<meta content="EN-US" name="DC.language" scheme="DCTERMS.RFC1766"/>
<meta content="text/html;charset=iso-8859-1" name="DC.format"/>
<meta content="1996-11-08" name="DC.date.created" scheme="ISO8601"/>
<meta content="2024-02-06" name="DC.date.reviewed" scheme="ISO8601"/>
<meta content=

In [34]:
# Find the element containing the text "Standard Meteorological"
element = soup.find('b', string="Standard Meterological")  

bouy_data_links = dict()

# Get the parent of the element
parent = element.parent 
liem = parent.find('ul')

# Print the parent element (or its contents)
for x in liem.find_all('li'):
    
    # print(x.text.split(':')[0])
    # print(x.find_all('a'))
    # for y in x.children:
    #     print(y.href)
    #     print(y.text)
    bouy_id = x.text.split(':')[0]
    links = list()
    for y in x.find_all('a'):
        link = 'https://www.ndbc.noaa.gov/view_text_file.php?' + y['href'].split('?')[-1]
        year = y.text
        links.append((year, link))
    bouy_data_links[bouy_id] = links
        # https://www.ndbc.noaa.gov/view_text_file.php?filename=18ci3h2015.txt.gz&dir=data/historical/stdmet/

In [36]:
len(bouy_data_links)

1315

In [37]:
import pandas as pd

In [40]:
pd.read_xml('https://www.ndbc.noaa.gov/metadata/stationmetadata.xml')

,id,name,owner,pgm,type,history
0,0Y2W3,"Sturgeon Bay CG Station, WI",U.S.C.G. Marine Reporting Stations,IOOS Partners,fixed,NaN
1,1801583,"SD 1063 - 24 NM SSW of San Francisco, CA (Site...",NDBC,NDBC Meteorological/Ocean,other,NaN
2,18CI3,"Michigan City CG Station, IN",U.S.C.G. Marine Reporting Stations,IOOS Partners,fixed,NaN
3,20CM4,"St. Joseph CG Station, MI",U.S.C.G. Marine Reporting Stations,IOOS Partners,fixed,NaN
4,21346,"21346 380km East of Iwate, Japan",Japanese Meteorological Agency,Tsunami,dart,NaN
...,...,...,...,...,...,...
1469,YGNN6,"Niagara Coast Guard Station, NY",NWS Eastern Region,IOOS Partners,fixed,NaN
1470,YKRV2,"8637611 - York River East Rear Range Light, VA",NOAA NOS PORTS,NOS/CO-OPS,fixed,NaN
1471,YKTV2,"8637689 - Yorktown USCG Training Center, VA",NOS,NOS/CO-OPS,fixed,NaN
1472,YRSV2,"Taskinas Creek, Chesapeake Bay Reserve, VA",National Estuarine Research Reserve System,NERRS,fixed,NaN


In [42]:
import requests

# URL of the JSON data
url = "https://www.ndbc.noaa.gov/ndbcmapstations.json"

# Make a GET request to the URL
response = requests.get(url)

# Get the JSON data from the response
data = response.json()

# Print the JSON data
print(data.keys())

dict_keys(['disclaimer', 'created', 'count', 'program', 'owner', 'station', 'dartevents'])


In [55]:
data['dartevents']

{'46410': '20250128085500'}

In [44]:
sd  = pd.DataFrame(data['station'])

In [45]:
sd

,id,lat,lon,elev,name,owner,program,status,data,type,seq
0,0y2w3,44.794,-87.313,179.0,"Sturgeon Bay CG Station, WI",106,2,E,n,fixed,NaN
1,13001,12.000,-23.000,0.0,NE Extension,84,1,E,y,buoy,NaN
2,13002,21.000,-23.000,0.0,NE Extension,84,1,E,y,buoy,NaN
3,13008,15.000,-38.000,0.0,Reggae,84,1,E,y,buoy,NaN
4,13009,8.000,-38.000,0.0,Lambada,84,1,E,n,buoy,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1899,ygnn6,43.262,-79.064,73.0,"Niagara Coast Guard Station, NY",82,2,E,y,fixed,NaN
1900,ykrv2,37.251,-76.342,NaN,"8637611 - York River East Rear Range Light, VA",73,5,D,n,fixed,NaN
1901,yktv2,37.227,-76.479,3.7,"8637689 - Yorktown USCG Training Center, VA",78,5,E,y,fixed,NaN
1902,yrsv2,37.414,-76.712,11.0,"Taskinas Creek, Chesapeake Bay Reserve, VA",64,4,E,y,fixed,NaN


In [50]:
sd['owner'] = sd.apply(lambda x: data['owner'][x.owner], axis=1)

In [52]:
sd['program'] = sd.apply(lambda x: data['program'][x.program], axis=1)

In [53]:
sd

,id,lat,lon,elev,name,owner,program,status,data,type,seq
0,0y2w3,44.794,-87.313,179.0,"Sturgeon Bay CG Station, WI",U.S.C.G. Marine Reporting Stations,IOOS Partners,E,n,fixed,NaN
1,13001,12.000,-23.000,0.0,NE Extension,Prediction and Research Moored Array in the At...,International Partners,E,y,buoy,NaN
2,13002,21.000,-23.000,0.0,NE Extension,Prediction and Research Moored Array in the At...,International Partners,E,y,buoy,NaN
3,13008,15.000,-38.000,0.0,Reggae,Prediction and Research Moored Array in the At...,International Partners,E,y,buoy,NaN
4,13009,8.000,-38.000,0.0,Lambada,Prediction and Research Moored Array in the At...,International Partners,E,n,buoy,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1899,ygnn6,43.262,-79.064,73.0,"Niagara Coast Guard Station, NY",NWS Eastern Region,IOOS Partners,E,y,fixed,NaN
1900,ykrv2,37.251,-76.342,NaN,"8637611 - York River East Rear Range Light, VA",NOAA NOS PORTS,NOS/CO-OPS,D,n,fixed,NaN
1901,yktv2,37.227,-76.479,3.7,"8637689 - Yorktown USCG Training Center, VA",NOS,NOS/CO-OPS,E,y,fixed,NaN
1902,yrsv2,37.414,-76.712,11.0,"Taskinas Creek, Chesapeake Bay Reserve, VA",National Estuarine Research Reserve System,NERRS,E,y,fixed,NaN
